<a href="https://colab.research.google.com/github/ChapelFob80930/AI-Generated-Text-Detector/blob/main/AITextTest2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

train_prompts = pd.read_csv("/content/train_prompts.csv")
print(train_prompts.head())
print("\n")
train_essays = pd.read_csv("/content/train_essays.csv")
print(train_essays.head())
print("\n")
test_essays = pd.read_csv("/content/test_essays.csv")
print(test_essays.head())
ai_human_dataset = pd.read_csv("/content/AI_Human.csv")
print(ai_human_dataset.head())

   prompt_id                       prompt_name  \
0          0                   Car-free cities   
1          1  Does the electoral college work?   

                                        instructions  \
0  Write an explanatory essay to inform fellow ci...   
1  Write a letter to your state senator in which ...   

                                         source_text  
0  # In German Suburb, Life Goes On Without Cars ...  
1  # What Is the Electoral College? by the Office...  


         id  prompt_id                                               text  \
0  0059830c          0  Cars. Cars have been around since they became ...   
1  005db917          0  Transportation is a large necessity in most co...   
2  008f63e3          0  "America's love affair with it's vehicles seem...   
3  00940276          0  How often do you ride in a car? Do you drive a...   
4  00c39458          0  Cars are a wonderful thing. They are perhaps o...   

   generated  
0          0  
1          0  
2    

ParserError: Error tokenizing data. C error: EOF inside string starting at row 41253

In [ ]:

import nltk
nltk.download()


NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all


       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package averaged_perceptron_tagger to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger.zip.
       | Downloading package averaged_perceptron_tagger_eng to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
       | Downloading package averaged_perceptron_tagger_ru to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_ru.zip.
       | Downloading package averaged_perceptron_tagger_rus to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_rus.zip.
       | Downloading package basque_grammars to /root/nltk_data...
       |   Unzipping grammars/basque_grammars.zip.
       | Downloading package bcp47 to /root/nltk_d


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

def lemmatize_text(text):
  doc = nlp(" ".join(text))
  return [token.lemma_ for token in doc]

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def _remove_stopwords_(texts):
  return [text for text in texts if text.isalnum() and text not in stop_words]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')

def preprocess_text(series):
    def process(text):
        tokens = word_tokenize(text)
        tokens = _remove_stopwords_(tokens)
        tokens = lemmatize_text(tokens)
        return " ".join(tokens)
    return series.apply(process)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
train_essays["preprocessedText"]=preprocess_text(train_essays['text'])

In [ ]:
print(train_essays.head())

         id  prompt_id                                               text  \
0  0059830c          0  Cars. Cars have been around since they became ...   
1  005db917          0  Transportation is a large necessity in most co...   
2  008f63e3          0  "America's love affair with it's vehicles seem...   
3  00940276          0  How often do you ride in a car? Do you drive a...   
4  00c39458          0  Cars are a wonderful thing. They are perhaps o...   

   generated                                   preprocessedText  
0          0  car Cars around since become famous 1900s Henr...  
1          0  transportation large necessity country worldwi...  
2          0  America love affair vehicle seem cool say Elis...  
3          0  how often ride car do drive one motor vehicle ...  
4          0  car wonderful thing they perhaps one world gre...  


In [ ]:
from gensim.models import Word2Vec

sentences = train_essays['preprocessedText'].apply(lambda x: x.split())  # Split each text into words

word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

word_vector = word2vec_model.wv['car']
print("Vector for 'car':", word_vector)

train_essays['word2vecVector'] = sentences.apply(
    lambda x: sum([word2vec_model.wv[word] for word in x if word in word2vec_model.wv]) / len(x)
)


Vector for 'car': [ 0.97536117 -0.35386756 -0.7083654   1.7872559  -0.2130904  -2.0731335
  2.8216224  -0.11210603 -0.8707031   0.73941773  0.64050037  1.6324179
  0.570496    1.2407081  -0.83913445 -1.6447902   0.7343392  -1.1417664
 -1.8130494  -2.1454384   2.4013557  -0.36828798  0.13669047  0.01100212
 -0.229631   -1.3351777  -0.54669    -0.11492027 -1.3314719  -0.26769474
 -0.9770468   0.11306044  0.06722984  0.26914147  0.765302    0.17495814
  0.5138002  -1.0157661   0.7645234  -2.237437   -0.6451902   0.83666396
 -1.144194    0.67588896 -0.08673283  0.401542   -0.18122281  0.3600087
  2.7873418   1.5840635  -0.10343429  2.010471    0.45882577  0.54794043
  0.62343913 -1.1196169   0.4397417  -0.5316869  -2.0372353   0.60499805
  1.109752   -0.6352352   0.10979827 -1.8308289  -1.2271293   1.9084932
  0.24928077  0.4578807  -1.0618341   0.3738086   0.7681737   0.4877699
  0.4547467  -2.1620193   1.7580478  -0.82683533  2.244563    0.6261877
 -1.3232875  -0.03577498  0.11127171  1.